# Hamming weight extractor

- Extracts the Hamming weight of the state after the SBox in the first round.
- Appends a hamming weight column to the ASCAD file.

In [1]:
import h5py
from src.data.loaders.ascad import ASCADData, ASCAD_DATA, ASCAD_DATA_VAR, ASCADDataType
from tqdm import tqdm
import numpy as np

from src.data.preprocess.hw import full_states

ascad = ASCADData.random_key()
plaintext_ix = 0
full_key_ix = 1

pm = ascad["Profiling_traces"]["metadata"]
am = ascad["Attack_traces"]["metadata"]

def get_acc(metadata):
    return np.zeros((len(metadata), 16), dtype=np.uint8)

profile_key, profile_plain = get_acc(pm), get_acc(pm)
attack_key, attack_plain = get_acc(am), get_acc(am)

for ix in tqdm(range(len(pm))):
    profile_key[ix] = pm[ix][full_key_ix]
    profile_plain[ix] = pm[ix][plaintext_ix]

for ix in tqdm(range(len(am))):
    attack_key[ix] = am[ix][full_key_ix]
    attack_plain[ix] = am[ix][plaintext_ix]

ascad.close()

  3%|▎         | 5290/200000 [00:01<00:51, 3776.78it/s]


KeyboardInterrupt: 

In [ ]:
profile_hw = full_states(profile_plain, profile_key)
attack_hw = full_states(attack_plain, attack_key)

In [ ]:
np.mean(profile_hw), np.mean(attack_hw)

In [ ]:
wrt = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r+')

for grp_name, grp_data in [("Profiling_traces", profile_hw), ("Attack_traces", attack_hw)]:
    grp = wrt[grp_name]

    if "hamming_weights" in grp:
        del grp["hamming_weights"]
    grp["hamming_weights"] = grp_data

wrt.close()

chk = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r')
print(list(chk["Profiling_traces"]))
print(list(chk["Attack_traces"]))
chk.close()

In [ ]:
ascad = ASCADData.random_key()
ascad["Profiling_traces"]["hamming_weights"][0]